In [2]:
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name='bigdefence/llama-3-Bllossom-unsloth-finetune'
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128009)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [11]:
from transformers import TextStreamer
def generate_response(model, tokenizer, messages, generation_config):
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    generation_config["streamer"] = streamer

    print("\n모델 응답:")
    with torch.no_grad():
        _ = model.generate(input_ids, **generation_config)


system_prompt = """You are a helpful AI assistant. Please answer the user's questions kindly.
당신은 유능한 AI 어시스턴트입니다. 사용자의 질문에 대해 친절하게 답변해주세요."""

eos_token_id = tokenizer.eos_token_id
eot_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

generation_config = {
    "max_new_tokens": 1024,
    "do_sample": True,
    "temperature": 0.6,
    "top_p": 0.95,
    "eos_token_id": [eos_token_id, eot_token_id],
}

instruction = 'KakaoVX의 잠재적인 타겟 고객층을 설명해주세요'

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": instruction}
]

generate_response(model, tokenizer, messages, generation_config)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



모델 응답:
assistant

KakaoVX의 잠재적인 타겟 고객층은 다음과 같습니다:

1. 일반 소비자: VR/AR 게임, 엔터테인먼트 콘텐츠 사용자
2. 기업: VR/AR 기술을 업무나 마케팅에 활용하고자 하는 기업
3. 교육기관: VR/AR 기반 교육 프로그램을 도입하고자 하는 학교, 대학, 교육센터
4. 의료기관: VR/AR 기술을 치료나 훈련에 활용하고자 하는 병원, 재활센터
5. 엔터테인먼트 업계: VR/AR 콘텐츠를 제작하고자 하는 영화, 음악, 공연 관련 기업
6. 부동산 및 건설 업계: VR 부동산 투어, AR 건축 설계 등을 활용하고자 하는 기업
7. 연구기관: VR/XR 기술 연구 및 개발을 위한 협력 파트너


In [12]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
"""
# 종료 토큰 설정
eos_token_id = tokenizer.eos_token_id
eot_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

generation_config = {
    "max_new_tokens": 1024,
    "do_sample": True,
    "temperature": 0.6,
    "top_p": 0.95,
    "eos_token_id": [eos_token_id, eot_token_id],
}
def generate_response(model, tokenizer, instruction, generation_config):
    # 파인튜닝 시 사용한 프롬프트 형식 적용
    prompt = alpaca_prompt.format(instruction)
    inputs = tokenizer(prompt, return_tensors="pt")
    # 모델 디바이스 확인
    inputs = inputs.to(model.device)

    # 스트리머 설정
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    generation_config["streamer"] = streamer

    print("\n모델 응답:")
    with torch.no_grad():
        outputs = model.generate(**inputs, **generation_config)

    # 생성된 텍스트 출력
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)



# 사용자 입력 받기
instruction = 'KakaoVX의 잠재적인 타겟 고객층을 설명해주세요'

generate_response(model, tokenizer, instruction, generation_config)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



모델 응답:
KakaoVX의 잠재적인 타겟 고객층은 다음과 같습니다:
1. 일반 소비자: VR/AR 게임, 엔터테인먼트 콘텐츠 사용자
2. 기업: VR/AR 기술을 업무나 마케팅에 활용하고자 하는 기업
3. 교육기관: VR/AR 기반 교육 프로그램을 도입하고자 하는 학교, 대학, 교육센터
4. 의료기관: VR/AR 기술을 치료나 훈련에 활용하고자 하는 병원, 재활센터
5. 엔터테인먼트 업계: VR/AR 콘텐츠를 제작하고자 하는 영화, 음악, 공연 관련 기업
6. 부동산 및 건설 업계: VR 부동산 투어, AR 건축 설계 등을 활용하고자 하는 기업
7. 연구기관: VR/XR 기술 연구 및 개발을 위한 협력 파트너
